In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import coint
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Basic configuration
DATE_CONFIG = {
    'TRAIN_START': pd.Timestamp('2021-02-02'),
    'TRAIN_END': pd.Timestamp('2024-02-02'),
    'TEST_END': pd.Timestamp('2025-01-01'),
    'TRADING_DAYS_PER_YEAR': 252  
}

def get_training_period():
    return {
        'start': DATE_CONFIG['TRAIN_START'],
        'end': DATE_CONFIG['TRAIN_END']
    }

def get_test_period():
    return {
        'start': DATE_CONFIG['TRAIN_END'],
        'end': DATE_CONFIG['TEST_END']
    }

def get_training_days():
    years = (DATE_CONFIG['TRAIN_END'] - DATE_CONFIG['TRAIN_START']).days / 365
    return int(years * DATE_CONFIG['TRADING_DAYS_PER_YEAR'])

# Plot settings
plt.style.use('classic')
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100

In [ ]:
def load_and_prepare_data(file_path):

    df = pd.read_parquet(file_path)
    df['date'] = pd.to_datetime(df['date'])
    
    mask = (df['date'] >= DATE_CONFIG['TRAIN_START']) & \
           (df['date'] <= DATE_CONFIG['TEST_END'])
    df = df[mask]
    
    price_matrix = df.pivot(index='date', columns='symbol', values='close')
    
    symbols = price_matrix.columns.tolist()
    
    print(f"Loaded data from {DATE_CONFIG['TRAIN_START']} to {DATE_CONFIG['TEST_END']}")
    print(f"Total symbols: {len(symbols)}")
    print(f"Total trading days: {len(price_matrix)}")
    
    return price_matrix, symbols

In [ ]:
def find_cointegrated_pairs(price_matrix, pvalue_threshold=0.01):
    n = price_matrix.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    symbols = price_matrix.columns
    pairs = []
    results = []
    
    total_pairs = sum(range(n))
    with tqdm(total=total_pairs, desc="Analyzing pairs") as pbar:
        for i in range(n):
            for j in range(i+1, n):
                S1 = price_matrix[symbols[i]]
                S2 = price_matrix[symbols[j]]
                
                result = coint(S1, S2)
                score = result[0]
                pvalue = result[1]
                
                score_matrix[i, j] = score
                pvalue_matrix[i, j] = pvalue
                
                results.append({
                    'symbol1': symbols[i],
                    'symbol2': symbols[j],
                    'p_value': pvalue,
                    'score': score
                })
                
                if pvalue <= pvalue_threshold:
                    pairs.append((symbols[i], symbols[j]))
                    
                pbar.update(1)
    
    return score_matrix, pvalue_matrix, pairs, results

def analyze_pairs(price_matrix, pvalue_threshold=0.01):
    """
    Runs the cointegration analysis and prints summary
    """
    score_matrix, pvalue_matrix, pairs, results = find_cointegrated_pairs(
        price_matrix, 
        pvalue_threshold
    )
    
    print(f"\nAnalysis complete!")
    print(f"Found {len(pairs)} cointegrated pairs")
    print(f"Total pairs analyzed: {len(results)}")
    
    summary_df = pd.DataFrame(results)
    summary_df['is_cointegrated'] = summary_df['p_value'] <= pvalue_threshold
    
    return score_matrix, pvalue_matrix, pairs, summary_df

def plot_cointegration_heatmap(pvalue_matrix, symbols, max_pvalue=0.98):
    """
    Creates a heatmap visualization of the cointegration p-values
    """
    plt.figure(figsize=(12, 8))
    mask = (pvalue_matrix >= max_pvalue)
    
    sns.heatmap(
        pvalue_matrix, 
        xticklabels=symbols, 
        yticklabels=symbols, 
        cmap='RdYlGn_r',
        mask=mask
    )
    
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.title('Cointegration p-values heatmap')
    plt.tight_layout()
    plt.show()

In [ ]:
price_matrix, symbols = load_and_prepare_data('nasdaq_daily.parquet')

score_matrix, pvalue_matrix, pairs, summary_df = analyze_pairs(
    price_matrix,
    pvalue_threshold=0.01
)

plt.figure(figsize=(10, 5))
plt.hist(summary_df['p_value'], bins=50)
plt.title('Distribution of Cointegration p-values')
plt.xlabel('p-value')
plt.ylabel('Frequency')
plt.show()

plot_cointegration_heatmap(pvalue_matrix, symbols)

print("\nTop 10 most cointegrated pairs (lowest p-values):")
top_pairs = summary_df.nsmallest(10, 'p_value')
print(top_pairs[['symbol1', 'symbol2', 'p_value']].to_string())

print(f"\nSummary Statistics:")
print(f"Total pairs analyzed: {len(summary_df)}")
print(f"Cointegrated pairs (p <= 0.01): {summary_df['is_cointegrated'].sum()}")
print(f"Average p-value: {summary_df['p_value'].mean():.4f}")
print(f"Median p-value: {summary_df['p_value'].median():.4f}")

In [ ]:
def zscore(series):
    return (series - series.mean()) / np.std(series)

def calculate_spread(data, symbol1, symbol2, start_date=None, end_date=None):
    if start_date:
        mask = (data.index >= start_date) & (data.index <= end_date)
        data = data[mask]
    
    # Calculate ratio and z-score
    ratios = data[symbol1] / data[symbol2]
    zscore_ratios = zscore(ratios)
    
    return ratios, zscore_ratios

def trade(S1_train, S2_train, S1_test, S2_test, window1=5, window2=60):
    ratios_train = S1_train / S2_train
    ma1_train = ratios_train.rolling(window=window1, center=False).mean()
    ma2_train = ratios_train.rolling(window=window2, center=False).mean()
    std_train = ratios_train.rolling(window=window2, center=False).std()

    # Calculate ratios for test period
    ratios_test = S1_test / S2_test
    
    # Initialize with last values from training
    last_ma1 = ma1_train.iloc[-1]
    last_ma2 = ma2_train.iloc[-1]
    last_std = std_train.iloc[-1]
    
    trades = []
    position = 0  # -1: short, 0: neutral, 1: long
    
    # Trading simulation
    for i in range(len(ratios_test)):
        current_ratio = ratios_test.iloc[i]
        current_date = ratios_test.index[i]
        
        # Calculate rolling mean and std in the test period
        ma1_test = ratios_test.iloc[:i+1].rolling(window=window1, center=False).mean().iloc[-1]
        ma2_test = ratios_test.iloc[:i+1].rolling(window=window2, center=False).mean().iloc[-1]
        std_test = ratios_test.iloc[:i+1].rolling(window=window2, center=False).std().iloc[-1]
        
        # Calculate z-score for the current ratio
        zscore = (current_ratio - ma2_test) / std_test
        
        trade_info = {
            'date': current_date,
            'zscore': zscore,
            'ratio': current_ratio,
            'ma1': ma1_test,
            'ma2': ma2_test,
            'S1_price': S1_test.iloc[i],
            'S2_price': S2_test.iloc[i],
            'position': position
        }
        
        if position == 0:  
            if zscore > 1.0:  
                trade_info['action'] = 'SHORT'
                position = -1
            elif zscore < -1.0:  
                trade_info['action'] = 'LONG'
                position = 1
            else:
                trade_info['action'] = 'HOLD'
                
        else:  # In position
            if abs(zscore) < 0.5:  # Exit condition
                trade_info['action'] = 'EXIT'
                position = 0
            else:
                trade_info['action'] = 'HOLD'
        
        trades.append(trade_info)

    return pd.DataFrame(trades)


def backtest_pair(price_matrix, symbol1, symbol2):
    training_mask = price_matrix.index < DATE_CONFIG['TRAIN_END']
    
    S1_train = price_matrix[symbol1][training_mask]
    S2_train = price_matrix[symbol2][training_mask]
    S1_test = price_matrix[symbol1][~training_mask]
    S2_test = price_matrix[symbol2][~training_mask]
    
    trades_df = trade(S1_train, S2_train, S1_test, S2_test)
    trades_df['pair'] = f"{symbol1}-{symbol2}"
    
    return trades_df

In [12]:
all_trades = []
for pair in pairs:
    symbol1, symbol2 = pair
    trades_df = backtest_pair(price_matrix, symbol1, symbol2)
    
    # Returns berechnen
    trades_df['returns'] = 0.0  # Initialisiere returns
    
    # Entry und Exit Preise für Positionen
    entry_price_s1 = None
    entry_price_s2 = None
    
    # Für jede Position die Returns berechnen
    for i in range(1, len(trades_df)):
        if trades_df['action'].iloc[i] in ['LONG', 'SHORT']:  # Position eröffnet
            entry_price_s1 = trades_df['S1_price'].iloc[i]
            entry_price_s2 = trades_df['S2_price'].iloc[i]
            
        elif trades_df['action'].iloc[i] == 'EXIT' and entry_price_s1 is not None:  # Position geschlossen
            if trades_df['position'].iloc[i-1] == 1:  # War Long position
                s1_return = ((trades_df['S1_price'].iloc[i] - entry_price_s1) / entry_price_s1)
                s2_return = ((trades_df['S2_price'].iloc[i] - entry_price_s2) / entry_price_s2)
                trades_df.loc[trades_df.index[i], 'returns'] = s1_return - s2_return
                
            else:  # War Short position
                s1_return = ((entry_price_s1 - trades_df['S1_price'].iloc[i]) / entry_price_s1)
                s2_return = ((trades_df['S2_price'].iloc[i] - entry_price_s2) / entry_price_s2)
                trades_df.loc[trades_df.index[i], 'returns'] = s1_return + s2_return
                
            entry_price_s1 = None
            entry_price_s2 = None
    
    all_trades.append(trades_df)

final_trades = pd.concat(all_trades)

# Performance Statistiken
print("\nPerformance Statistics:")
print(f"Total Return: {final_trades['returns'].sum():.2%}")
print(f"Average Trade Return: {final_trades[final_trades['returns'] != 0]['returns'].mean():.2%}")
print(f"Return Std Dev: {final_trades[final_trades['returns'] != 0]['returns'].std():.2%}")
print(f"Sharpe Ratio: {(final_trades['returns'].mean() / final_trades['returns'].std() * np.sqrt(252)):.2f}")
print(f"Win Rate: {(final_trades['returns'] > 0).sum() / (final_trades['returns'] != 0).sum():.2%}")

# Per pair performance
pair_performance = final_trades.groupby('pair')['returns'].agg([
    ('total_return', 'sum'),
    ('avg_trade_return', lambda x: x[x != 0].mean()),
    ('win_rate', lambda x: (x > 0).sum() / (x != 0).sum())
]).sort_values('total_return', ascending=False)

print("\nTop 5 Performing Pairs:")
print(pair_performance.head().to_string())
print("\nBottom 5 Performing Pairs:")
print(pair_performance.tail().to_string())



Performance Statistics:
Total Return: 507.70%
Average Trade Return: 1.78%
Return Std Dev: 9.43%
Sharpe Ratio: 0.49
Win Rate: 78.67%

Top 5 Performing Pairs:
           total_return  avg_trade_return  win_rate
pair                                               
CSX-ZS         0.475789          0.047579  0.900000
CSX-DDOG       0.403246          0.050406  1.000000
KHC-ODFL       0.392908          0.098227  1.000000
AMAT-NXPI      0.384209          0.042690  1.000000
AVGO-BKNG      0.367120          0.052446  0.857143

Bottom 5 Performing Pairs:
          total_return  avg_trade_return  win_rate
pair                                              
KHC-TTD      -0.146963         -0.036741  0.500000
KDP-MNST     -0.174543         -0.058181  0.000000
CSX-QCOM     -0.234722         -0.033532  0.428571
EA-MSTR      -0.314134         -0.044876  0.857143
MSTR-TSM     -0.440482         -0.088096  0.800000
